**Download sample datasets**

In [1]:
!rm -rf sample_data
!wget --no-check-certificate https://download.smartappli.eu/medium.tar
!mkdir FIRE
!tar -xvf medium.tar -C FIRE
!rm medium.tar

--2023-04-30 08:32:03--  https://download.smartappli.eu/medium.tar
Resolving download.smartappli.eu (download.smartappli.eu)... 46.105.57.169, 2001:41d0:301::20
Connecting to download.smartappli.eu (download.smartappli.eu)|46.105.57.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2528903168 (2.4G) [application/x-tar]
Saving to: ‘medium.tar’

medium.tar          100%[===================>]   2.35G  11.8MB/s    in 3m 28s  

2023-04-30 08:35:32 (11.6 MB/s) - ‘medium.tar’ saved [2528903168/2528903168]

medium/
medium/fire/
medium/fire/10-9-15-2-400.jpg
medium/fire/10.png
medium/fire/100.png
medium/fire/101.png
medium/fire/102.png
medium/fire/103.png
medium/fire/104.png
medium/fire/105.png
medium/fire/106.png
medium/fire/107.png
medium/fire/108.png
medium/fire/109.png
medium/fire/11.png
medium/fire/110.png
medium/fire/111.png
medium/fire/112.png
medium/fire/113.png
medium/fire/114.png
medium/fire/115.png
medium/fire/116.png
medium/fire/117.png
medium/fire/118.pn

**Load libraries**

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms

from torch.utils.data import DataLoader, SubsetRandomSampler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

**Train a fire model (AlexNet)**


In [9]:
batch_size	= 8
train_split = 0.7
epochs		= 30
random		= torch.Generator().manual_seed(42)
criterion   = nn.CrossEntropyLoss()
optimizer	= optim.Adam #optim.SGD
learn_rate	= 0.001


transform   = transforms.Compose([
						transforms.RandomResizedCrop(224),
						transforms.ToTensor()
					])
dataset   = datasets.ImageFolder('FIRE/medium',transform=transform)
indices		= list(range(len(dataset)))
# shuffle
shuffle     = torch.randint(len(dataset),(len(dataset),),generator=random)
for (i,j) in enumerate(shuffle):
	indices[i],indices[j] = indices[j],indices[i]
del shuffle #clean

trainloader = DataLoader(
						dataset,
						batch_size=batch_size,
						sampler=SubsetRandomSampler(indices[:int(len(indices)*train_split)])
					)

valloader = DataLoader(
						dataset,
						batch_size=batch_size,
						sampler=SubsetRandomSampler(indices[int(len(indices)*train_split):])
					)

fire_model = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.DEFAULT)

fire_model.classifier[1] = nn.Linear(in_features=1792, out_features=3, bias=True)

fire_model.to(device)

optimizer   = optimizer(fire_model.parameters(), lr=learn_rate)

for epoch in range(epochs):
	fire_model.train(True)
	running_loss = 0.0
	counter = 0
	for (i, data) in enumerate(trainloader):
		inputs, labels = data
		labels = torch.nn.functional.one_hot(torch.LongTensor(labels),num_classes=len(dataset.classes)).float()
		inputs, labels = inputs.to(device), labels.to(device)
		# zero the parameter gradients
		optimizer.zero_grad()
		# forward + backward + optimize
		outputs = fire_model(inputs)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		# print statistics
		running_loss += loss.item()
		counter += 1
	print(f'train [{epoch + 1}, {i + 1:5d}] loss: {running_loss / counter:.3f}')
	fire_model.train(False)
	running_loss = 0.0
	counter = 0
	for (i, data) in enumerate(valloader):
		inputs, labels = data
		inputs, labels = inputs.to(device), labels.to(device)
		# forward
		outputs = fire_model(inputs)
		loss = criterion(outputs, labels)
		# print statistics
		running_loss += loss.item()
		counter += 1
		torch.save(fire_model, 'firenet-%d.pt' % epoch)
	print(f'val [{epoch + 1}, {i + 1:5d}] loss: {running_loss / counter:.3f}')
torch.save(fire_model, 'firenet.pt')


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

KeyboardInterrupt: ignored

In [6]:
#torch.save(fire_model,'firenet.pt')